In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import os

In [ ]:
arr = data.X.toarray()
# non zero values of the first row in arr
nonzero = arr[0][arr[0].nonzero()]
print(nonzero)
print(len(nonzero))

[ 1.        1.        1.       ... 21.000002  7.       24.000002]
2547


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc


## Count ADlasso features

In [3]:
# adata = sc.read_h5ad('/home/jovyan/work/Research_datasets/Hao_PBMC.h5ad')
# adata = sc.read_h5ad("C:\Research_datasets\Hao_PBMC.h5ad")
adata = sc.read_h5ad("C:\Research_datasets\Hao_PBMC_GSE164378.h5ad")
adata.obs['celltype.l2'] = [s.replace(' ', '_') for s in adata.obs['celltype.l2']]
print(adata.shape)

label = adata.obs['celltype.l2'].tolist()
types = np.unique(label).tolist()
print('all cell types:', types)

(161764, 33538)
all cell types: ['ASDC', 'B_intermediate', 'B_memory', 'B_naive', 'CD14_Mono', 'CD16_Mono', 'CD4_CTL', 'CD4_Naive', 'CD4_Proliferating', 'CD4_TCM', 'CD4_TEM', 'CD8_Naive', 'CD8_Proliferating', 'CD8_TCM', 'CD8_TEM', 'Doublet', 'Eryth', 'HSPC', 'ILC', 'MAIT', 'NK', 'NK_CD56bright', 'NK_Proliferating', 'Plasmablast', 'Platelet', 'Treg', 'cDC1', 'cDC2', 'dnT', 'gdT', 'pDC']


In [4]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
print(adata.shape)


(161764, 33538)


In [7]:
# os.chdir('/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao/Level2_pvl0_ttsplit/lambda_decision_ignore_section/L2_feature_selection')
os.chdir(r"C:\Users\evanlee\Documents\GitHub\EvanPys\Progress\PBMC_Hao\Level2_pvl0_ttsplit\lambda_decision_ignore_section\L2_feature_selection")

features_dict = {}
# Read features for each celltype
for celltype in types:
    try:
        feature_df = pd.read_csv(f'{celltype}_features.txt', names=['Gene', 'Weight', 'Tendency'], sep='\t')
        features_dict[celltype] = feature_df
    except:
        print('skipping:', celltype)
        continue
    # print(celltype, 'Feature count:', feature_df.shape[0])
    # print(celltype, 'Positive feature count:', feature_df[feature_df['Tendency'] == 1].shape[0])
    # print('------------------')

count_df = pd.DataFrame(columns=['Feature_count', 'Positive_feature_count'])
for celltype in features_dict.keys():
    feature_df = features_dict[celltype]
    feature_count = feature_df.shape[0]
    positive_count = feature_df[feature_df['Tendency'] == 1].shape[0]
    count_df.loc[celltype] = [feature_count, positive_count]
count_df.head()

,Feature_count,Positive_feature_count
ASDC,10,6
B_intermediate,11,8
B_memory,20,12
B_naive,21,12
CD14_Mono,15,7


In [8]:
count_df.shape

(31, 2)

## Build DEGn classifier for each celltype

In [9]:
# Find marker genes
def rank_genes(adata, all_types):
    sc.tl.rank_genes_groups(adata, groupby='celltype.l2', n_genes=2000, method='wilcoxon')

    # Get the top ranked genes for each celltype
    genes_df_dict = {}

    for celltype in all_types:
        genes_df = sc.get.rank_genes_groups_df(adata, group=celltype)
        genes_df_dict[celltype] = genes_df
    
    return genes_df_dict
    

In [10]:
DE_genes_dict = rank_genes(adata, types)

c:\Users\evanlee\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\evanlee\AppData\Local\Programs\Python\Python39\lib\site-packages\scanpy\tools\_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
c:\Users\evanlee\AppData\Local\Programs\Python\Python39\lib\site-packages\scanpy\tools\_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is 

In [11]:
DE_genes_dict.keys()

dict_keys(['ASDC', 'B_intermediate', 'B_memory', 'B_naive', 'CD14_Mono', 'CD16_Mono', 'CD4_CTL', 'CD4_Naive', 'CD4_Proliferating', 'CD4_TCM', 'CD4_TEM', 'CD8_Naive', 'CD8_Proliferating', 'CD8_TCM', 'CD8_TEM', 'Doublet', 'Eryth', 'HSPC', 'ILC', 'MAIT', 'NK', 'NK_CD56bright', 'NK_Proliferating', 'Plasmablast', 'Platelet', 'Treg', 'cDC1', 'cDC2', 'dnT', 'gdT', 'pDC'])

In [12]:
DE_genes_dict['ASDC']

,names,scores,logfoldchanges,pvals,pvals_adj
0,PLD4,14.798680,9.658385,1.493857e-49,5.010096e-45
1,PPP1R14A,14.719638,11.169179,4.822386e-49,8.086659e-45
2,TCF4,14.433711,8.924783,3.175281e-47,3.549752e-43
3,ITM2C,14.282030,9.059283,2.832298e-46,2.374740e-42
4,SAMHD1,14.209684,4.327047,7.978480e-46,5.351645e-42
...,...,...,...,...,...
1995,SHARPIN,3.075433,1.918811,2.101975e-03,3.204365e-02
1996,PBX2,3.074683,1.904725,2.107263e-03,3.210967e-02
1997,PIP4P1,3.073300,1.988577,2.117057e-03,3.224425e-02
1998,DHRS4L2,3.069284,1.909616,2.145726e-03,3.266607e-02


In [13]:

celltype = 'ASDC'
n_features = count_df.loc[celltype, 'Feature_count']
DEGn = DE_genes_dict[celltype]['names'][:n_features].tolist()

# subset adata to only include DEGs
# adata_DEGn = adata[:, DEGn]


# construct LR classifier
# subset data to DEGs
X = adata[:, DEGn].X
print(f'{celltype} DEGn adata shape:', X.shape)
# Binary label
y = [1 if i==celltype else 0 for i in adata.obs['celltype.l2'].tolist()]



ASDC DEGn adata shape: (161764, 10)


In [14]:
DEGn

['PLD4',
 'PPP1R14A',
 'TCF4',
 'ITM2C',
 'SAMHD1',
 'CCDC50',
 'IRF8',
 'LILRA4',
 'CST3',
 'IL3RA']

In [15]:
def LR_classifier(data, DEGn, celltype):
    print('==================')
    print('Constructing classifier for:', celltype)
    # subset data to DEGs
    X = data[:, DEGn].X
    print(X.shape)
    # Binary label
    y = [1 if i==celltype else 0 for i in data.obs['celltype.l2'].tolist()]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    
    # train classifier
    clf = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0)
    clf.fit(X_train, y_train)

    # evaluate classifier
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    precision = precision_score(y_test, y_pred)  # average='macro'
    print("Precision: {:.2f}%".format(precision * 100))
    f1 = f1_score(y_test, y_pred)
    print("F1: {:.2f}%".format(f1 * 100))
    # AUC-ROC
    y_pred_prob = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    print('ROC-AUC:', roc_auc)

    # model metrics
    metrics = [accuracy, precision, f1, roc_auc]

    return clf, metrics, y_test, y_pred

In [16]:
clf_AS, metrics_AS, y_t, y_p = LR_classifier(adata, DEGn, celltype)

Constructing classifier for: ASDC
(161764, 10)
Accuracy: 99.98%
Precision: 85.71%
F1: 82.76%
ROC-AUC: 0.9999567072793618


In [17]:
np.unique(y_t)

array([0, 1])

In [18]:
np.unique(y_p)

array([0, 1])

In [22]:
!pwd

/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao/Level2_pvl0_ttsplit/lambda_decision_ignore_section/L2_feature_selection


In [23]:
os.chdir('/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao/Classifier_evaluation')

In [24]:
# # export DEG list
# for celltype in DE_genes_dict.keys():
#     DEG_df = DE_genes_dict[celltype]
#     DEG_df.to_csv(f'{celltype}_DEG2000.csv')